Часто изучая новый для себя алгоритм не перестаешь удивляться - как до этого раньше никто не додумался? Часто идея, лежащая в основе прорывной научной статьи, кажется лежащей на поверхности. Так же получается и с алгоритмом градиентного бустинга.

Градиентный бустинг - это одна из разновидностей бустинга, поэтому сначала мы рассмотрим бустинг в целом, а затем перейдем непосредственно к градиентному бустингу.

## Что такое бустинг?

Бустинг - это способ построения ансамбля, в котором обучается много копий более слабой модели ("weak learner"). Как правило этой моделью является решающее дерево. На каждом шаге новый weak learner концентрируется на исправлении ошибок, допущенных предыдущими weak learner'ами. В итоге предсказания всех weak learner'ов суммируются с определенными весами. Бустинг чем-то похож на бэггинг, но в бэггинге weak learner'ы обучаются совершенно независимо и параллельно, а в бустинге последовательно, с оглядкой на предыдущие. Такой подход - обучение множества "слабых" алгоритмов - противоположен другому подходу, в котором обучается один "сильный" алгоритм.

<img src="assets/flexair.gif" width="400" align="left">

## Ранний вариант бустинга: AdaBoost

Итак, в бустинге каждый следующий weak learner стремится скорректировать предсказания предыдущих. Это можно сделать разными способами. Одной из первых эффективных реализаций бустинга был AdaBoost - в нем каждый следующий weak learner фокусировал внимание на тех примерах, на которых предыдущие weak learner'ы дали неверные ответы. При этом он не знал, какие именно ответы даны предыдущими weak learner'ами - было лишь известно, что они неверны. Задачей нового weak learner'а было дать верные ответы преимущественно на этих примерах.

Заметим, что при этом не используется никакого валидационного датасета. Используется только обучающий датасет, на нем же оценивается точность предыдущих weak learner'ов. Это означает, что если очередной weak learner после обучения дал верные ответы на все примеры, то бустинг продолжить будет невозможно. Например, если в качестве weak learner'а мы используем решающее дерево неограниченной глубины, то так и произойдет. Нужно использовать решающие деревья небольшой глубины: weak learner должен быть действительно "слабым", не переобучаясь слишком сильно.

## Градиентный бустинг

В градиентном бустинге целевыми данными для следующего weak learner'а является *градиент (со знаком минус) функции потерь по предсказаниям предыдущих алгоритмов*. Таким образом следующий weak learner корректирует предсказания предыдущих.

Например, производная функции потерь $MSE(y, \hat{y}) = (y - \hat{y})^2$ по $\hat{y}$ равна $2(\hat{y} - y)$, а значит предсказывая градиент weak learner как раз и предсказывает разность предсказания и правильного ответа. Но можно использовать и другие функции потерь.

Далее рассмотрим все описанное более формально.

## Постановка задачи и обозначения

Для дальнейшего изложения введем необходимые обозначения:

- **Обучающий датасет** состоит из массива исходных данных $\textbf{X} = [X_1, X_2, ..., X_n]$ и массива эталонных ответов $\textbf{y} = [y_1, y_2, ..., y_n]$. Каждый $X_i$ представляет собой набор признаков, в простом случае - вектор фиксированной длины. В задаче классификации будем считать, что $\textbf{y}$ представлен в one-hot кодировании.

- **Обучаемый алгоритм** имеет параметры $\theta$, принимает на вход исходные данные и возвращает предсказания: $f(X_i, \theta) = \hat{y_i}$. Массив предсказаний алгоритма на всем обучающем датасете обозначим как $\hat{\mathbf{y}} = f(\textbf{X}, \theta)$. Количество параметров может быть как фиксированным (например, нейронная сеть), так и переменным (например, растущий случайный лес). Предсказание может быть, а может не быть дифференцируемым по параметрам $\theta$. Для *градиентного спуска* требуется дифференцируемость по параметрам, для *градиентного бустинга* - не требуется.

- **Функция потерь** принимает на вход эталонный ответ и предсказание и выдает число: $loss(y_i, \hat{y_i})$. Мы хотим минимизировать сумму значений функции потерь по всему обучающему датасету: $sum\_loss(\textbf{y}, \hat{\mathbf{y}}) = \sum_{i=1}^{n} loss(y_i, \hat{y_i})$. Как для *градиентного спуска*, так и для *градиентного бустинга* требуется дифференцируемость функции потерь по предсказаниям.

## Алгоритм градиентного бустинга

Пусть мы имеем обучающий датасет, обучаемый алгоритм (weak learner) и функцию потерь.

В случае **регрессии** weak learner'ом как правило является решающее дерево. Более интересен случай **классификации**: чтобы считать функцию потерь нам необходимо получать предсказания в виде logit'ов, то есть набора действительных чисел. Поэтому weak learner'ом в данном случае является сразу несколько решающих деревьев: по одному на каждый класс.

Выберем константу $C$ так, чтобы сумма $\sum_{i=1}^n loss(y_i, C)$ была минимальна. Эта константа будет нашим начальным приближением, к ней мы будем прибавлять предсказания weak learner'ов.

Заранее выберем число шагов $N$.

**for k = 0, ..., N-1:**

1. На $k$-м шаге мы уже обучили $k$ weak learner'ов. Мы получаем предсказания с помощью их взвешенной суммы для всех примеров из обучающего датасета:

 $\hat{y}_i = С + \sum_{j=0}^k w_i f(X_i, \theta_j)$

2. Считаем функцию потерь для каждого предсказания: $loss(y_i, \hat{y}_i)$

3. Считаем производную функции потерь со знаком минус по каждому предсказанию: $r_i = -\frac{\partial}{\partial \hat{y}_i} loss(y_i, \hat{y}_i)$. Таким образом мы получаем информацию о том, как нам нужно изменить каждое предсказание, чтобы функция потерь уменьшилась (исходя из смысла понятия производной).

4. Обучаем новый weak learner предсказывать $r_i$ по $X_i$. Обозначим параметры нового weak learner'а за $\theta_k$.

5. Осталось выбрать вес для нового weak learner'а. Для этого получаем предсказания нового weak learner'а на всем обучающем датасете: $\hat{r}_i = f(X_i, \theta_k)$. Затем подбираем такой вес $w_k$, чтобы значение $sum\_loss(\mathbf{y}, \hat{\mathbf{y}} + w_k*\hat{\mathbf{r}})$ было минимально.

В итоге мы получаем ансамбль из $N$ weak learner'ов. Алгоритм инференса (то есть предсказания на произвольных данных) выглядит аналогично:

 $\hat{y} = С + \sum_{j=0}^N w_i f(X, \theta_j)$

## Связь с градиентным спуском

Идея как градиентного спуска, так и градиентного бустинга состоит в том, что мы *рассчитываем градиент функции потерь по предсказаниям*, а затем ходим сдвинуть предсказания в направлении, противоположном градиенту, и таким образом сделать их более точными.

Но в градиентном спуске это достигается с помощью распространения градиента на веса и обновления весов, а в градиентном бустинге с помощью прибавления предсказаний нового weak learner'а, который аппроксимирует градиент со знаком минус. Таким образом, в градиентном спуске используется фиксированное число параметров, а в градиентном бустинге - переменное (каждый новый weak learner содержит новые параметры).

Иногда градиентный бустинг рассматривают как покоординатный градиентный спуск в пространстве функций.

## Обсуждение

Как уже упоминалось выше, в случае выбора функции потерь $MSE$ ее производная пропорциональна разности предсказания и верного ответа, поэтому градиентный бустинг сводится к тому, что каждый следующий weak learner пытается предсказать величину ошибки, допущенную предыдущим. Но при этом сам допускает ошибки, и их пытается предсказать уже следующий weak learner, и так далее.

Важно, чтобы weak learner не был способен слишком сильно переобучиться, иначе следующим weak learner'ам не на чем будет учиться. Поэтому в качестве weak learner'а обычно выбирают деревья небольшой глубины, которые имеют низкую выразительную способность. Линейная регрессия не подходит в качестве weak learner'а, поскольку взвешенная сумма линейных моделей линейна, поэтому и весь ансамбль получится линейным.

## Выбор функции потерь

Интересно, что если мы выберем в градиентном бустинге специальную "экспоненциальную" функцию потерь (*поля этой книги слишком узки для записи ее формулы*), то мы получаем алгоритм, эквивалентный AdaBoost. Конечно с вычистельной точки зрения эти алгоритмы получатся все равно разные, но было доказано, что они эквивалентны в плане получаемого результата.

Таким образом, алгоритм AdaBoost эквивалентен частному случаю градиентного бустинга.

## Регуляризация градиентного бустинга

Для того, чтобы ослабить переобучение градиентного бустинга, применяются следующие техники:

**Subsampling.** Мы обучаем каждый следующий weak learner не на всем датасете, а на случайной подвыборке. В этом случае градиентный бустинг называется стохастическим. В sklearn гиперпараметр `subsample` определяет какая доля обучающего датасета будет использоваться при построении каждого следующего дерева. В sklearn подвыборки выбираются без возвращения (в отличие от бэггинга), то есть один и тот же пример не может попасть в выборку 2 раза.

**Shrinkage.** После того, как мы расчитали вес нового weak learner'а $w_k$, мы умножаем его на некоторое число меньше 1. Таким образом мы укорачиваем шаг в направлении градиента. В sklearn этот гиперпараметр называется `learning_rate`. Чем меньше значение `learning_rate`, тем медленнее будет сходиться градиентный бустинг, то есть потребуется больше шагов, но переобучение будет ниже. Иногда рекомендуют использовать значение `learning_rate` меньше 0.1.

**Регуляризация weak learner'а.** Например, в sklearn можно использовать параметр `max_features`, который определяет, сколько случайно выбранных признаков будет использоваться при поиске оптимального разбиения в каждом узле дерева.